In [1]:
from sympy import *

其中的一些公式参考 https://www.math.uwaterloo.ca/~hwolkowi/matrixcookbook.pdf

# 抽象类型和具体类型

在 SymPy 中，可以声明抽象的符号/向量/矩阵，也可以填入数据，声明具体的符号/向量/矩阵

In [5]:
x = symbols('x')
m, n = symbols('m n')
y = MatrixSymbol('y', n, 1)
Z = MatrixSymbol('Z', m, n)
display(x, y, Z)

x

y

Z

In [6]:
x = 5
y = Matrix([1, 2, 3])
Z = Matrix([[1, 2], [3, 4]])
display(x, y, Z)

5

Matrix([
[1],
[2],
[3]])

Matrix([
[1, 2],
[3, 4]])

具体矩阵的求导会返回矩阵或者多维数组的形式

而抽象矩阵的求导会以符号方式进行运算

# 纯符号运算（完善）

如果是向量和矩阵之间的符号计算，可以用 `MatrixSymbol` 实现抽象矩阵的定义，利用抽象的矩阵符号进行求导

然后再使用 `as_explicit` 展开，或者用 `subs` 替换成具体值

## $\dfrac{\partial{\textbf x^\top \textbf a}}{\partial{\textbf x}}=\textbf a$

In [2]:
m, n = symbols('m n')
x = MatrixSymbol('x', m, 1)
a = MatrixSymbol('a', m, 1)

In [3]:
expr = x.T * a
expr

x.T*a

In [4]:
d = diff(expr, x, evaluate = False)
d

Derivative(x.T*a, x)

In [5]:
d = d.doit()
d

a

得到了一个抽象的 $\textbf a$，而不是具体大小的向量

## $\dfrac{\partial{\textbf a^\top \textbf X \textbf b}}{\partial{\textbf X}}=\textbf a \textbf b^\top$

矩阵的大小可以像上面一样写成符号

这里指定了具体的数，才能可视化打印出来

In [6]:
X = MatrixSymbol('X', 2, 3) # 'X', m, n
a = MatrixSymbol('a', 2, 1) # 'a', m, 1
b = MatrixSymbol('b', 3, 1) # 'X', n, 1

In [7]:
expr = a.T * X * b
expr

a.T*X*b

In [8]:
d = diff(expr, X, evaluate = False)
d

Derivative(a.T*X*b, X)

In [9]:
d = d.doit()
d

PermuteDims(ArrayTensorProduct(a, b), (3)(1 2))

这里的 `ArrayTensorProduct(a, b)` 就是 a 和 b 的 Tensor 乘法，得到一个 (2, 1, 3, 1) 的数组

相当于 $ab^\top$

然后使用 `PermuteDims` 将数组整理成 (2, 3, 1, 1)

In [10]:
d.as_explicit()

[[[[a[0, 0]*b[0, 0]]], [[a[0, 0]*b[1, 0]]], [[a[0, 0]*b[2, 0]]]], [[[a[1, 0]*b[0, 0]]], [[a[1, 0]*b[1, 0]]], [[a[1, 0]*b[2, 0]]]]]

用 `reshape` 转成二维矩阵

In [11]:
d.as_explicit().reshape(2,3)

[[a[0, 0]*b[0, 0], a[0, 0]*b[1, 0], a[0, 0]*b[2, 0]], [a[1, 0]*b[0, 0], a[1, 0]*b[1, 0], a[1, 0]*b[2, 0]]]

## $\dfrac{\partial{\textbf a^\top \textbf X^\top \textbf b}}{\partial{\textbf X}}=\textbf b \textbf a^\top$

In [12]:
X = MatrixSymbol('X', 3, 2) # 'X', n, m
a = MatrixSymbol('a', 2, 1) # 'a', m, 1
b = MatrixSymbol('b', 3, 1) # 'X', n, 1

In [13]:
expr = a.T * X.T * b
expr

a.T*X.T*b

In [14]:
d = diff(expr, X, evaluate = False)
d

Derivative(a.T*X.T*b, X)

In [15]:
d = d.doit()
d

PermuteDims(ArrayTensorProduct(b, a), (1 2 3))

这里变成了 `ArrayTensorProduct(b, a)`

相当于 $ba^\top$

和上面的 `ArrayTensorProduct(a, b)` 不一样了

In [16]:
d.as_explicit()

[[[[a[0, 0]*b[0, 0]]], [[a[1, 0]*b[0, 0]]]], [[[a[0, 0]*b[1, 0]]], [[a[1, 0]*b[1, 0]]]], [[[a[0, 0]*b[2, 0]]], [[a[1, 0]*b[2, 0]]]]]

用 `reshape` 转成二维矩阵

In [17]:
result = d.as_explicit().reshape(3,2)
result

[[a[0, 0]*b[0, 0], a[1, 0]*b[0, 0]], [a[0, 0]*b[1, 0], a[1, 0]*b[1, 0]], [a[0, 0]*b[2, 0], a[1, 0]*b[2, 0]]]

如果要代入具体的数值，使用 `subs` 来替换即可

In [18]:
a1 = Matrix([1,2])
b1 = Matrix([3,4,5])

In [19]:
result.subs({a:a1, b:b1})

[[3, 6], [4, 8], [5, 10]]

## $\dfrac{\partial(\textbf B\textbf x+\textbf b)^\top\textbf C(\textbf D\textbf x +\textbf d)}{\partial\textbf x} 
= \textbf B^\top\textbf C(\textbf D\textbf x+\textbf d)+\textbf D^\top\textbf C^\top(\textbf B\textbf x+ \textbf b)$

In [6]:
n, m, p = symbols('n m p')
x = MatrixSymbol('x', n, 1)
B = MatrixSymbol('B', m, n)
b = MatrixSymbol('b', m, 1)
C = MatrixSymbol('C', m, p)
D = MatrixSymbol('D', p, n)
d = MatrixSymbol('d', p, 1)

In [7]:
expr = (B * x + b).T * C * (D * x + d)
expr

(b.T + x.T*B.T)*C*(D*x + d)

In [8]:
r = diff(expr, x, evaluate = False)
r

Derivative((b.T + x.T*B.T)*C*(D*x + d), x)

In [9]:
r = r.doit()
r

B.T*C*(D*x + d) + D.T*C.T*(B*x + b)

# 不完善的情况

## $\dfrac{\partial\textbf X}{\partial X_{ij}}=\textbf J^{ij}$（矩阵内的元素）

遇到矩阵内特定元素的求导，需要指定具体大小，用 `as_explicit` 展开再求导

In [23]:
X = MatrixSymbol('X', 2, 3)
display(X)
display(X[1,1])

X

X[1, 1]

直接求导没办法得到正确的结果

In [16]:
diff(X, X[1,1])

0

这种情况必须要先展开再求导

但是如果矩阵的大小不确定，例如 `MatrixSymbol('X', m, n)`，则没有办法展开

或者元素的位置不确定，例如 `X[m, n]`，则得到全 0 的矩阵

In [24]:
X = X.as_explicit()
X

Matrix([
[X[0, 0], X[0, 1], X[0, 2]],
[X[1, 0], X[1, 1], X[1, 2]]])

In [25]:
diff(X, X[1,1])

Matrix([
[0, 0, 0],
[0, 1, 0]])

## $\dfrac{\partial\textbf a^\top \textbf X^n \textbf b}{\partial \textbf X}$ $=\sum_{r=0}^{n-1} (\textbf X^r)^\top \textbf a\textbf b^\top(\textbf X^{n-1-r})^\top$（矩阵的幂）

矩阵的幂的求导也有问题，必须指定具体的次数

In [26]:
n, m, p, q = symbols('n m p q')
X = MatrixSymbol('X', 2, 2)
a = MatrixSymbol('a', 2, 1)
b = MatrixSymbol('b', 2, 1)

In [35]:
expr = a.T * X**n * b
r1 = diff(expr, X, evaluate = False)
r1

Derivative(a.T*X**n*b, X)

直接用 n 求导是有问题的

In [36]:
r1 = r1.doit()
r1

n*DiagMatrix(a)*HadamardPower(X, n - 1)*DiagMatrix(b)

In [37]:
r1 = r1.subs(n, 3)
r1.as_explicit()

Matrix([
[3*X[0, 0]**2*a[0, 0]*b[0, 0], 3*X[0, 1]**2*a[0, 0]*b[1, 0]],
[3*X[1, 0]**2*a[1, 0]*b[0, 0], 3*X[1, 1]**2*a[1, 0]*b[1, 0]]])

要将次数 n 替换成具体的数才能正确计算

In [38]:
expr2 = expr.subs(n, 3)
r2 = diff(expr2, X, evaluate = False)
r2

Derivative(a.T*X**3*b, X)

In [39]:
r2 = r2.doit()
r2

X.T**2*a*b.T + a*b.T*X.T**2 + X.T*a*b.T*X.T

In [40]:
r2.as_explicit()

Matrix([
[((X[0, 0]*X[0, 1] + X[0, 1]*X[1, 1])*b[1, 0] + (X[0, 0]**2 + X[0, 1]*X[1, 0])*b[0, 0])*a[0, 0] + ((X[0, 0]*X[1, 0] + X[1, 0]*X[1, 1])*a[1, 0] + (X[0, 0]**2 + X[0, 1]*X[1, 0])*a[0, 0])*b[0, 0] + (X[0, 0]*a[0, 0] + X[1, 0]*a[1, 0])*(X[0, 0]*b[0, 0] + X[0, 1]*b[1, 0]), ((X[0, 0]*X[1, 0] + X[1, 0]*X[1, 1])*a[1, 0] + (X[0, 0]**2 + X[0, 1]*X[1, 0])*a[0, 0])*b[1, 0] + ((X[0, 0]*X[1, 0] + X[1, 0]*X[1, 1])*b[0, 0] + (X[0, 1]*X[1, 0] + X[1, 1]**2)*b[1, 0])*a[0, 0] + (X[0, 0]*a[0, 0] + X[1, 0]*a[1, 0])*(X[1, 0]*b[0, 0] + X[1, 1]*b[1, 0])],
[((X[0, 0]*X[0, 1] + X[0, 1]*X[1, 1])*a[0, 0] + (X[0, 1]*X[1, 0] + X[1, 1]**2)*a[1, 0])*b[0, 0] + ((X[0, 0]*X[0, 1] + X[0, 1]*X[1, 1])*b[1, 0] + (X[0, 0]**2 + X[0, 1]*X[1, 0])*b[0, 0])*a[1, 0] + (X[0, 0]*b[0, 0] + X[0, 1]*b[1, 0])*(X[0, 1]*a[0, 0] + X[1, 1]*a[1, 0]), ((X[0, 0]*X[0, 1] + X[0, 1]*X[1, 1])*a[0, 0] + (X[0, 1]*X[1, 0] + X[1, 1]**2)*a[1, 0])*b[1, 0] + ((X[0, 0]*X[1, 0] + X[1, 0]*X[1, 1])*b[0, 0] + (X[0, 1]*X[1, 0] + X[1, 1]**2)*b[1, 0])*a[1